# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследование:

   1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
   2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
   3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведения пользователей (статистика о платежеспособности клиентов) мы получаем из файла - '/datasets/data.csv'. О качестве данных ничего не известно. 
 Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на поставленные вопросы.
 4. Общий вывод.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
try:
    df=pd.read_csv('/datasets/data.csv')
except:
    df=pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
    
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце days_employed мы видим отрицательные значения и вещественный тип чисел,необходимо преобразование.

В столбце education возможно преобразование слов к одному регистру, для дальнейшей корректной работы с ним. 

In [3]:
df.describe()       #describe() используются для просмотра некоторых основных статистических данных

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обнаруживаем "интересные" значения, *children* детей не может быть "-1" , и конечно вряд ли может быть и "20". 

Скорее всего данные были введены некорретно. 

Преобразуем данные по модулю, и  возможно заменить значение "20" медианным, но в данном этапе проекте необходимо только условие, есть дети(от 1 и далее) или нет(0).

Возраст в некоторых строках *dob_years* равен 0, возможно пропущен или просто не указан.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. 

Тип данных в столбцах различен: float64(2), int64(5), object(5).

Мы можем увидеть,что количество значений в столбцах различаются. Значит, в данных есть пропущенные значения.


Согласно документации к данным:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита
    
   

**Вывод**

В каждой строке таблицы — данные о клиентах. Часть колонок описывает персональные данные клиентов: количество детей в семье, общий трудовой стаж, возраст, уровень образования, семейное положение,тип занятости, пол и ежемесячный доход. Остальные данные рассказывают о клиенте: имел ли он задолженность по возврату кредита и какова цель получения кредита. 

Предварительно можно утверждать, что данных достаточно для ответов на поставленные вопросы. Но мы встречаем пропуски в данных , а они могут непосредственно влиять на конечный результат.
Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Преобразуем столбец *education* к нижнему регистру.

In [5]:
df['education']=df['education'].str.lower()

Преобразуем столбец со стажем по модулю,чтобы избавиться от  отрицательных значений. Проверим результат.

In [6]:
df['days_employed']=df['days_employed'].abs()  #преобразование отрицательных значений 
print(df['days_employed'])

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


Проверим количество значений с отсутствующими данными.

In [7]:
print(df.isna().sum())      #cколько в таблице пропущенных значений и где они содержатся

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Проверим, это одни и те же строки, где отсутствуют данные по обоим столбцам ('days_employed' и 'total_income') или нет.

In [8]:
is_nan=df[df['days_employed'].isna() & df['total_income'].isna()]
display(is_nan.head(10))
print('Количество строк с пропусками:',is_nan.shape[0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Количество строк с пропусками: 2174


Предположение подтвердилось, в строках где отсутсвуют данные по столбцу 'days_employed' там же отсутствуют данные и по 'total_income'

Далее проанализируем как связаны отсутствующие данные по стажу *days_employed* и типу занятости *income_type*.

Для этого зададим условия нулевых значений по нашим столбцам и посчитаем количество клиентов относящихся к каждой группе (тип занятости)

In [9]:
df[(df['total_income'].isna() == True) & (df['days_employed'].isna() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Видим, что пропущенные данные из разных категорий, поэтому средним значением из одной категории заменять их не целесообразно.

Мы не можем однозначно сказать почему некоторые значения в столбцах days_employed (трудовой стаж) и total_income (ежемесячный доход) отсутствуют, возможно пользователи не захотели предоставить эту информацию, но скорее всего в оформлении заявки на кредит она была обязательна. 

Тогда возможно на появления пропущенных значений повлиял человеческих фактор или сбой при экспорте данных. В данной ситуации у нас есть достаточно много похожих показателей от других пользователей, поэтому мы можем заменить пропущенные значения средним значением общего трудого стажа клиента. 

In [10]:
df['days_employed']=df['days_employed'].fillna(value=df['days_employed'].mean())

Пропущенные значения в столбце total_income(ежемесяный доход) некорректно заменить средним показателем, потому как есть большие перепады в значениях. Оптимальный способ заменить пропущенные значения медианой (значение признака, находящееся в середине ранжированной (упорядоченной по возрастанию или убыванию) совокупности)

In [11]:
df['total_income']=df['total_income'].fillna(value=df['days_employed'].median())

Выведем значения медианы в зависимости от занятости клиентов

In [12]:
total_median = df.groupby('income_type')['total_income'].median()
print(total_median)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        139034.452386
компаньон          162401.351555
пенсионер          110179.690761
предприниматель    250886.492981
сотрудник          133546.457238
студент             98201.625314
Name: total_income, dtype: float64


In [13]:
mediana=df['total_income'].median()   #сравним с общей медианой по всей группе 
print(mediana)

135514.71128002706


Заменим пропущенные значения в столбце *total_income* на 0. Затем применим последовательно два условия (по нулевым значениям и типу занятости клиентов) и заменим нулевые на медианные значения по группам.

In [14]:
df['total_income'] = df['total_income'].fillna(0)

In [15]:
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'безработный '), 'total_income'] = total_median[0]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'в декрете'), 'total_income'] = total_median[1]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'госслужащий '), 'total_income'] = total_median[2]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'компаньон'), 'total_income'] = total_median[3]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'пенсионер'), 'total_income'] = total_median[4]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'предприниматель'), 'total_income'] = total_median[5]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'сотрудник'), 'total_income'] = total_median[6]
df.loc[(df['total_income'] == 0) & (df['income_type'] == 'студент'), 'total_income'] = total_median[7]



Другой вариант заполнения медианным значением используя словарь и метод замены пропусков *fillna*

In [16]:
df=df.fillna({'total_income':df.groupby('income_type')['total_income'].transform('median')}) 

Проверим наш датасет на отсутствие пропущенных значений в данных.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Провели преобработку пропущенных значений данных в столбцах (трудовой стаж и ежемесячный доход). 

Преобразовали пропущенные значения  заменив на среднее значение и медианное по группам соотвественно.

### Замена типа данных

В столбце days_employed (трудовой стаж) мы видим вещественный тип данных (float64), это некорректные данные , стаж может измеряться только в целых днях(int). 

Замена типа данных методом to_numeric() в данном случае нам не подойдет, так как при переводе все числа будут иметь тип данных float.

В нужный тип значения переводят методом astype().

In [18]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')#преобразование типа данных в целочисленный


Выведем обновленную информацию по нашему датасету и проверим произведенную замену типа данных.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Замена типа данных произведенна корректно.

### Обработка дубликатов

Теперь посчитаем общее количество дубликатов. 

Используем стандартный метод *duplicated()* выводим суммарное количество дубликатов по датафрейму:

In [20]:
df.duplicated().sum()

71

In [21]:
df[df.duplicated()].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,66914,41,среднее,1,женат / замужем,0,F,сотрудник,0,2609,покупка жилья для семьи
3290,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,2609,сыграть свадьбу
4182,1,66914,34,высшее,0,гражданский брак,1,F,сотрудник,0,2609,свадьба
4851,0,66914,60,среднее,1,гражданский брак,1,F,пенсионер,0,2609,свадьба
5557,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,2609,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,66914,64,среднее,1,женат / замужем,0,F,пенсионер,0,2609,дополнительное образование
21032,0,66914,60,среднее,1,женат / замужем,0,F,пенсионер,0,2609,заняться образованием
21132,0,66914,47,среднее,1,женат / замужем,0,F,сотрудник,0,2609,ремонт жилью
21281,1,66914,30,высшее,0,женат / замужем,0,F,сотрудник,0,2609,покупка коммерческой недвижимости


Похоже на задвоение данных

Удалим пропуски из таблицы методом dropna() и восстановите индексы методом reset_index() c параметром drop, равным True.

In [22]:
df['total_income']=df['total_income'].drop_duplicates()
df = df.dropna().reset_index(drop=True)

In [23]:
df['days_employed']=df['days_employed'].drop_duplicates()
df = df.dropna().reset_index(drop=True)

Проверим заданные таблицы на наличие дубликатов.

In [24]:
print ('Дубликатов в таблице days_employed:', df['days_employed'].duplicated().sum())

Дубликатов в таблице days_employed: 0


In [25]:
print ('Дубликатов в таблице total_income:', df['total_income'].duplicated().sum())

Дубликатов в таблице total_income: 0


**Вывод**

Дубликаты в данных устраненны.

### Лемматизация

Мы можем выделить леммы в значениях столбца с целями получения кредита. 

Для этого мы будем использовать библиотеку с функцией лемматизации на русском языке — pymystem3, разработана сотрудниками Яндекса.

In [26]:
from pymystem3 import Mystem                  #импортируем библиотеку pymystem3
m = Mystem() 

Выведем список уникальных целей

In [27]:
print(df['purpose'].value_counts())

свадьба                                   311
сыграть свадьбу                           310
на проведение свадьбы                     310
покупка жилья                             294
операции с недвижимостью                  276
покупка жилья для сдачи                   275
покупка коммерческой недвижимости         270
строительство жилой недвижимости          270
покупка недвижимости                      268
операции с коммерческой недвижимостью     266
недвижимость                              265
покупка своего жилья                      265
жилье                                     265
покупка жилья для семьи                   264
строительство недвижимости                262
операции с жильем                         261
строительство собственной недвижимости    260
операции со своей недвижимостью           256
ремонт жилью                              242
покупка жилой недвижимости                227
автомобиль                                216
на покупку подержанного автомобиля

Нам нужно получить основы слов для корректной работы с таблицей purpose.

In [28]:
m = Mystem()
list_lemmas = []
for index in df['purpose']:
    lemma = m.lemmatize(index)
    list_lemmas.append(lemma)
print(list_lemmas)

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилье', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['недвижимость', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'подержа

Выделим список с основными категориями.

Проверим лемматизацию, заменим полученный список лемм на слово из списка *lem_cat*

In [29]:
lem_cat = ['жилье','образование','свадьба','недвижимость','автомобиль']

def categoria(text):
    lemma = m.lemmatize(text)
    for index in lem_cat:
        if index in lemma:
            lemma = index
    return lemma


Проверим на корректность работы нашу функцию categoria.

In [30]:
print(categoria('покупка жилья'))

жилье


Преобразуем функцию *categoria(text)*

Выделим основные категории: *коммерческая недвижимость*, *автомобиль*,*свадьба*, *образование*.

In [31]:
def categoria(row):
    lemma = m.lemmatize(row)
    if ('жилье' or 'недвижимость') in lemma:
        return 'коммерческая недвижимость'
    elif 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif 'образование' in lemma:
        return 'образование'

Далее применим метод *apply()* c нашей функцией *сategoria*  к столбцу *purpose* 

In [32]:
df['purpose_group']=df['purpose'].apply(categoria)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.0,покупка жилья,коммерческая недвижимость
1,1,4024.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля,автомобиль
2,0,5623.0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.0,покупка жилья,коммерческая недвижимость
3,3,4124.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.0,дополнительное образование,образование
4,0,340266.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу,свадьба
5,0,926.0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.0,покупка жилья,коммерческая недвижимость
6,0,2879.0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.0,операции с жильем,коммерческая недвижимость
7,0,152.0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.0,образование,образование
8,2,6929.0,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.0,на проведение свадьбы,свадьба
9,0,2188.0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.0,покупка жилья для семьи,коммерческая недвижимость


**Вывод**

Лемматизация целей кредита произведена , в дальнейшем она приходится нам в категоризации данных.

### Категоризация данных

Категоризация по целям была проведена ранее при помощи лемматизации.
Сгруппируем по типу и количеству цели.


In [33]:
df['purpose_group'].value_counts()

коммерческая недвижимость    1866
автомобиль                   1759
образование                  1708
свадьба                       931
Name: purpose_group, dtype: int64

Чаще всего люди берут  кредиты именно для покупки жилья или недвижимости.

In [34]:
print(df['total_income'].value_counts())

327678.0    1
121512.0    1
189779.0    1
533870.0    1
77153.0     1
           ..
70924.0     1
162466.0    1
50468.0     1
64285.0     1
151552.0    1
Name: total_income, Length: 8884, dtype: int64


Можно произвести категоризацию по возрастной группе клиентов, заодно обработав нулевые значения в данной категории.

In [35]:
print(df['dob_years'].value_counts())

59    309
58    285
57    278
56    276
60    266
61    262
62    254
55    252
54    248
52    240
53    240
48    227
44    222
50    218
42    217
41    210
63    205
49    202
40    197
47    196
64    195
38    195
45    193
39    192
51    192
35    189
43    187
33    185
36    183
46    182
34    164
37    160
31    154
65    147
30    142
66    139
67    137
29    134
32    123
27    114
28    108
26     89
68     83
69     76
25     74
70     56
24     53
71     47
23     44
0      36
22     34
72     29
21     18
20     12
73      7
74      5
19      2
Name: dob_years, dtype: int64


Выявим какой тип занятости имеют клиенты с нулевым возрастом

In [36]:
df.loc[df['dob_years'] == 0, 'income_type'].value_counts()

пенсионер      14
сотрудник      12
компаньон       6
госслужащий     4
Name: income_type, dtype: int64

In [37]:
age_median = df.groupby('income_type')['dob_years'].median()
print(age_median)

income_type
безработный    38
в декрете      39
госслужащий    42
компаньон      42
пенсионер      60
сотрудник      42
Name: dob_years, dtype: int64


Заменим клиентов с нулевым значением возраста на средние значения в соответствии с типом категориями. 

Применим для этого логическую индексацию с несколькими условиями

In [38]:
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_median[5]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_median[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_median[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_median[2]

Проверим наличие нулевого возраста в нашем датафрейме

In [39]:
df.loc[df['dob_years'] == 0, 'dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

Теперь можем приступать к категоризации по возрастной группе.
Для этого напишем вспомогательную функцию *years*

In [40]:
def age_group(age):
    if age < 30:
        return 'до 30 лет'
    elif 30<=age < 45:
        return 'от 30 до 45'
    elif 45<=age<65:
        return 'от 45 до 65'
    elif age>=65:
         return 'после 65' 
df['age_group'] = df['dob_years'].apply(age_group)
print(df.head()) 

   children  days_employed  dob_years education  education_id  \
0         1         8437.0         42    высшее             0   
1         1         4024.0         36   среднее             1   
2         0         5623.0         33   среднее             1   
3         3         4124.0         32   среднее             1   
4         0       340266.0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0      253875.0   
1   женат / замужем                 0      F   сотрудник     0      112080.0   
2   женат / замужем                 0      M   сотрудник     0      145885.0   
3   женат / замужем                 0      M   сотрудник     0      267628.0   
4  гражданский брак                 1      F   пенсионер     0      158616.0   

                      purpose              purpose_group    age_group  
0               покупка жилья  коммерческая недвижимость

In [41]:
df['age_group'].value_counts()

от 45 до 65    4734
от 30 до 45    2742
после 65        726
до 30 лет       682
Name: age_group, dtype: int64

На основании категоризации данных по возрастной группе мы видим что больше всего людей в возрасте от 45 до 65 пользуются услугами по кредитованию

Далее мы можем категоризировать данные по наличию детей у клиентов.

In [42]:
def have_children(children):
    if children>=1:
        return "есть дети"
    if children==0:
        return "нет детей"
df['children_group'] = df['children'].apply(have_children)
print(df.head()) 

   children  days_employed  dob_years education  education_id  \
0         1         8437.0         42    высшее             0   
1         1         4024.0         36   среднее             1   
2         0         5623.0         33   среднее             1   
3         3         4124.0         32   среднее             1   
4         0       340266.0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0      253875.0   
1   женат / замужем                 0      F   сотрудник     0      112080.0   
2   женат / замужем                 0      M   сотрудник     0      145885.0   
3   женат / замужем                 0      M   сотрудник     0      267628.0   
4  гражданский брак                 1      F   пенсионер     0      158616.0   

                      purpose              purpose_group    age_group  \
0               покупка жилья  коммерческая недвижимост

In [43]:
df['children_group'].value_counts() 

нет детей    6427
есть дети    2439
Name: children_group, dtype: int64

Далее перейдем произведем категоризацию по уровню дохода.
Создадим вспомогательную функцию *salary*

In [44]:
def salary(row):
    if row<= 50000:
        return 'бедные'
    elif 50000 < row<= 120000:
        return 'средний класс'
    elif 120000 < row< 1000000:
        return 'богатые'
df['total_group'] = df['total_income'].apply(salary)
print(df.head())    
        

   children  days_employed  dob_years education  education_id  \
0         1         8437.0         42    высшее             0   
1         1         4024.0         36   среднее             1   
2         0         5623.0         33   среднее             1   
3         3         4124.0         32   среднее             1   
4         0       340266.0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0      253875.0   
1   женат / замужем                 0      F   сотрудник     0      112080.0   
2   женат / замужем                 0      M   сотрудник     0      145885.0   
3   женат / замужем                 0      M   сотрудник     0      267628.0   
4  гражданский брак                 1      F   пенсионер     0      158616.0   

                      purpose              purpose_group    age_group  \
0               покупка жилья  коммерческая недвижимост

In [45]:
df['total_group'].value_counts() 

богатые          5454
средний класс    3164
бедные            255
Name: total_group, dtype: int64

Богатые чаще всего берут кредиты, наверно чтобы пустить свои деньги в оборот и приумножить их.

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 16 columns):
children            8884 non-null int64
days_employed       8884 non-null float64
dob_years           8884 non-null int64
education           8884 non-null object
education_id        8884 non-null int64
family_status       8884 non-null object
family_status_id    8884 non-null int64
gender              8884 non-null object
income_type         8884 non-null object
debt                8884 non-null int64
total_income        8884 non-null float64
purpose             8884 non-null object
purpose_group       6264 non-null object
age_group           8884 non-null object
children_group      8866 non-null object
total_group         8873 non-null object
dtypes: float64(2), int64(5), object(9)
memory usage: 1.1+ MB


**Вывод**

Проведена категоризация клиентов по целям , возрастным группам и наличию детей.
**Дописать **

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим новую таблицу *children* с данными о наличии детей у клиентов и наличием задолженности по кредиту

In [47]:
children=df[['debt','children_group']]
print(children)

      debt children_group
0        0      есть дети
1        0      есть дети
2        0      нет детей
3        0      есть дети
4        0      нет детей
...    ...            ...
8879     0      нет детей
8880     0      нет детей
8881     0      нет детей
8882     0      нет детей
8883     0      нет детей

[8884 rows x 2 columns]


Зададим условие - не имел задолженность по кредиту и группируем по наличию детей нашу таблицу *children*

In [48]:
children_debt=children.loc[(children['debt'] == 0)]['children_group'].value_counts()
print(children_debt)

нет детей    6039
есть дети    2264
Name: children_group, dtype: int64


Далее мы должны посчитать соотношение клиентов не имеющих задолжности по кредиту к общему количеству клиентов не имеющих детей.

In [49]:
count_nonechildren=children['children_group'].value_counts()
print(count_nonechildren)

нет детей    6427
есть дети    2439
Name: children_group, dtype: int64


In [50]:
conv_free_child=round((children_debt[0]/count_nonechildren[0])*100,1)
conv_child=round((children_debt[1]/count_nonechildren[1])*100,1)
print('Процентное соотношение клиентов (без наличия детей) вернувших кредит в срок, составляет')
print(conv_free_child)
print('Процентное соотношение клиентов (имеющих детей) вернувших кредит в срок, составляет')
print(conv_child)

Процентное соотношение клиентов (без наличия детей) вернувших кредит в срок, составляет
94.0
Процентное соотношение клиентов (имеющих детей) вернувших кредит в срок, составляет
92.8


In [51]:
#children_debt_none=children.loc[(children['debt'] == 1)]['children_group'].value_counts()
#print(children_debt) ДАННЫЕ О НАЛИЧИИ ДЕТЕЙ ПРИ НЕВОЗВРАТЕ КРЕДИТА

In [52]:
#conv_free_child=round((children_debt_none[0]/count_nonechildren[0])*100,1)
#conv_child=round((children_debt_none[1]/count_nonechildren[1])*100,1)
#print('Процентное соотношение клиентов (без наличия детей) невернувших кредит в срок, составляет')
#print(conv_free_child)
#print('Процентное соотношение клиентов (имеющих детей) невернувших кредит в срок, составляет')
#print(conv_child)

**Вывод**

Мы можем сделать вывод что клиенты у которых нет детей возращают чуть лучше кредиты в срок банку. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим новую таблицу *family_debt* в которой будут данные клиентов о их семейном положении и возвратностью кредитов в срок

In [53]:
family_debt=df[['family_status','debt']]
print(family_debt)

         family_status  debt
0      женат / замужем     0
1      женат / замужем     0
2      женат / замужем     0
3      женат / замужем     0
4     гражданский брак     0
...                ...   ...
8879  гражданский брак     0
8880   женат / замужем     0
8881   женат / замужем     0
8882   женат / замужем     0
8883   женат / замужем     0

[8884 rows x 2 columns]


Выявим все уникальные значения в таблице семейного положения клиентов

In [54]:
family_debt['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Создадим новую переменную *count_family* в которой будут хранится данные о всех пользователях в соответствии с их семейным положением

In [55]:
count_family=family_debt['family_status'].value_counts()
print(count_family)

женат / замужем          5210
гражданский брак         1582
Не женат / не замужем     940
вдовец / вдова            632
в разводе                 520
Name: family_status, dtype: int64


В таблице *family* информация о возврате кредита с срок в соответствии с семейным положением клиентов

In [56]:
family=family_debt.loc[(family_debt['debt'] == 0)]['family_status'].value_counts()
print(family)

женат / замужем          4898
гражданский брак         1461
Не женат / не замужем     876
вдовец / вдова            592
в разводе                 494
Name: family_status, dtype: int64


Посчитаем процентое соотношение возвратности кредита в срок по каждой группе семейного положения клиентов

In [57]:
conv_married=round((family[0]/count_family[0])*100,1)
conv_civil_mar=round((family[1]/count_family[1])*100,1)
conv_notmar=round((family[2]/count_family[2])*100,1)
conv_widow=round((family[3]/count_family[3])*100,1)
conv_divors=round((family[4]/count_family[4])*100,1)
print('Процентное соотношение клиентов (женат/замужем) вернувших кредит в срок, составляет')
print(conv_married)
print('Процентное соотношение клиентов (гражданский брак) вернувших кредит в срок, составляет')
print(conv_civil_mar)
print('Процентное соотношение клиентов (не женат/не замужем) вернувших кредит в срок, составляет')
print(conv_notmar)
print('Процентное соотношение клиентов (вдовец/вдова) вернувших кредит в срок, составляет')
print(conv_widow)
print('Процентное соотношение клиентов (в разводе) вернувших кредит в срок, составляет')
print(conv_divors)

Процентное соотношение клиентов (женат/замужем) вернувших кредит в срок, составляет
94.0
Процентное соотношение клиентов (гражданский брак) вернувших кредит в срок, составляет
92.4
Процентное соотношение клиентов (не женат/не замужем) вернувших кредит в срок, составляет
93.2
Процентное соотношение клиентов (вдовец/вдова) вернувших кредит в срок, составляет
93.7
Процентное соотношение клиентов (в разводе) вернувших кредит в срок, составляет
95.0


**Вывод**

Интересный вывод , в разводе клиенты лучше возвращают кредиты. 


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим таблицу *salary_debt* зависимости возвратности кредита и уровня дохода клиентов по категории

In [58]:
salary_debt=df[['debt','total_group']]
print(salary_debt)

      debt    total_group
0        0        богатые
1        0  средний класс
2        0        богатые
3        0        богатые
4        0        богатые
...    ...            ...
8879     0  средний класс
8880     0  средний класс
8881     0  средний класс
8882     0        богатые
8883     0        богатые

[8884 rows x 2 columns]


Вычислим данные по условию возвратности кредита в срок и уровню доходности клиентов.

In [59]:
salary=salary_debt.loc[(salary_debt['debt'] == 0)]['total_group'].value_counts()
print(salary)

богатые          5088
средний класс    2981
бедные            242
Name: total_group, dtype: int64


Данные об общем количестве клиентов по уровню доходности

In [60]:
count_salary=salary_debt['total_group'].value_counts()
print(count_salary)

богатые          5454
средний класс    3164
бедные            255
Name: total_group, dtype: int64


In [61]:
conv_rich=round((salary[0]/count_salary[0])*100,1)
conv_avg=round((salary[1]/count_salary[1])*100,1)
conv_poor=round((salary[2]/count_salary[2])*100,1)

print('Процентное соотношение богатых клиентов вернувших кредит в срок, составляет')
print(conv_rich)
print('Процентное соотношение клиентов среднего класса вернувших кредит в срок, составляет')
print(conv_avg)
print('Процентное соотношение бедных клиентов вернувших кредит в срок, составляет')
print(conv_poor)


Процентное соотношение богатых клиентов вернувших кредит в срок, составляет
93.3
Процентное соотношение клиентов среднего класса вернувших кредит в срок, составляет
94.2
Процентное соотношение бедных клиентов вернувших кредит в срок, составляет
94.9


**Вывод**

Как ни странно бедные клиенты более обязательны в возратах кредита

- Как разные цели кредита влияют на его возврат в срок?

Создадим таблицу *purpose_debt* зависимостей целей получения кредита и задолженности по кредиту

In [62]:
purpose_debt=df[['debt','purpose_group']]
print(purpose_debt)

      debt              purpose_group
0        0  коммерческая недвижимость
1        0                 автомобиль
2        0  коммерческая недвижимость
3        0                образование
4        0                    свадьба
...    ...                        ...
8879     0                    свадьба
8880     0                       None
8881     0                       None
8882     0                 автомобиль
8883     0                 автомобиль

[8884 rows x 2 columns]


Создадим новую переменную *count_purpose* в которой будут хранится данные о всех пользователях в соответствии с их целями получения кредита.

In [63]:
count_purpose=purpose_debt['purpose_group'].value_counts()
print(count_purpose)

коммерческая недвижимость    1866
автомобиль                   1759
образование                  1708
свадьба                       931
Name: purpose_group, dtype: int64


Получим данные по целям получения кредита с его возвратностью в срок.

In [64]:
purpose=purpose_debt.loc[(purpose_debt['debt'] == 0)]['purpose_group'].value_counts()
print(purpose)

коммерческая недвижимость    1760
автомобиль                   1627
образование                  1587
свадьба                       873
Name: purpose_group, dtype: int64


In [65]:
conv_realty=round((purpose[0]/count_purpose[0])*100,1)
conv_avto=round((purpose[1]/count_purpose[1])*100,1)
conv_educ=round((purpose[2]/count_purpose[2])*100,1)
conv_wedd=round((purpose[3]/count_purpose[3])*100,1)
print('Процентное соотношение клиентов с целью- коммерческая недвижимость вернувших кредит в срок, составляет')
print(conv_realty)
print('Процентное соотношение клиентов с целью-автомобиль вернувших кредит в срок, составляет')
print(conv_avto)
print('Процентное соотношение клиентов с целью-образование вернувших кредит в срок, составляет')
print(conv_educ)
print('Процентное соотношение клиентов с целью-свадьба вернувших кредит в срок, составляет')
print(conv_wedd)

Процентное соотношение клиентов с целью- коммерческая недвижимость вернувших кредит в срок, составляет
94.3
Процентное соотношение клиентов с целью-автомобиль вернувших кредит в срок, составляет
92.5
Процентное соотношение клиентов с целью-образование вернувших кредит в срок, составляет
92.9
Процентное соотношение клиентов с целью-свадьба вернувших кредит в срок, составляет
93.8


**Вывод**

Лучше всего клиенты возвращают кредиты по недвижимости/жилью.

## Шаг 4. Общий вывод

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке. 
Наличие детей у клиентов влияет на возврат кредита в срок, бездетные реже просрачивают оплату.

Построим сводную таблицу, для ответа на вопрос: 

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [66]:
children_debt=df.loc[:,['children','children_group','debt']]
children_debt.head(10)

,children,children_group,debt
0,1,есть дети,0
1,1,есть дети,0
2,0,нет детей,0
3,3,есть дети,0
4,0,нет детей,0
5,0,нет детей,0
6,0,нет детей,0
7,0,нет детей,0
8,2,есть дети,0
9,0,нет детей,0


In [67]:
#создадим сводную таблицу 
children_pivot=df.pivot_table(
    index='children_group',
    columns='debt',
    values='children',
    aggfunc='count')
children_pivot.head(10)

debt,0,1
children_group,,
есть дети,2264,175
нет детей,6039,388


In [68]:
#создадим сводную таблицу 
children_pivot=children_debt.pivot_table(
    index='children_group',
    columns='debt',
    values='children',
    aggfunc='count')
children_pivot.head(10)

debt,0,1
children_group,,
есть дети,2264,175
нет детей,6039,388


In [69]:
children_pivot['percent']=(children_pivot[0]/(children_pivot[0]+children_pivot[1]))*100
children_pivot

debt,0,1,percent
children_group,,,
есть дети,2264,175,92.824928
нет детей,6039,388,93.962969


Клиенты без детей возвращают кредиты все же лучше.

**Исследуем вопрос возврастности кредита в срок в соответствии с возрастной группой**

In [70]:
age_debt=df.loc[:,['dob_years','age_group','debt']]
age_debt.head()

,dob_years,age_group,debt
0,42,от 30 до 45,0
1,36,от 30 до 45,0
2,33,от 30 до 45,0
3,32,от 30 до 45,0
4,53,от 45 до 65,0


In [71]:
#создадим сводную таблицу 
age_pivot=age_debt.pivot_table(
    index='age_group',
    columns='debt',
    values='dob_years',
    aggfunc='count')
age_pivot.head(10)

debt,0,1
age_group,,
до 30 лет,618,64
от 30 до 45,2530,212
от 45 до 65,4484,250
после 65,689,37


Добавим к сводной таблице столбец по процентной задолженности в соответствии с возрастной группой 

In [72]:
age_pivot['percent']=(age_pivot[0]/(age_pivot[0]+age_pivot[1]))*100
age_pivot

debt,0,1,percent
age_group,,,
до 30 лет,618,64,90.615836
от 30 до 45,2530,212,92.268417
от 45 до 65,4484,250,94.719054
после 65,689,37,94.903581


Ответственней всего проявляют себя люди пенсионного возраста. Конечно в силу возраста люди могут быть более ответственны.

Возможно их суммы кредита невелики и они быстрей погашают свои кредиты,но это лишь наши предположения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.